In [2]:
pip install geopy


  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 125.4/125.4 kB 7.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00


In [3]:
import geopy
print(geopy.__version__)

2.4.1


In [4]:
pip install requests beautifulsoup4 geopy pandas census

  Obtaining dependency information for census from https://files.pythonhosted.org/packages/99/73/3868a695f082f379dce20f19b55451fef4c3f4337824f0991dc1a228301b/census-0.8.24-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install requests beautifulsoup4 geopy pandas census

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests  # Imports library for HTTP API calls
from geopy.geocoders import Nominatim  # Imports Nominatim for geocoding
from geopy.extra.rate_limiter import RateLimiter  # Imports RateLimiter for request throttling
import pandas as pd  # Imports pandas for data handling
from census import Census  # Imports Census for ACS5 data access
import time  # Imports time for timing operations
import os  # Imports os for system interactions
import matplotlib.pyplot as plt  # Imports matplotlib for visualizations

# Constants  # Defines constant values
CENSUS_API_KEY = "86cfe7a999f6607864204747a5ac83b7c77e02fb"  # Census API key
RADIUS = 5  # Radius in miles for OSM queries
BASE_YEAR = 2021  # Base year for Census data

geolocator = Nominatim(user_agent="impact_tool")  # Initializes Nominatim with user agent
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=5)  # Rate-limits geocoding
c = Census(CENSUS_API_KEY, year=BASE_YEAR)  # Sets up Census client
census_cache = {}  # Empty dict for caching Census data

def geocode_address(address):  # Converts address to coordinates
    location = geocode(address, timeout=10)  # Geocodes with 10s timeout
    if location:  # Checks if geocoding succeeded
        return location.latitude, location.longitude  # Returns lat/lon tuple
    raise ValueError("Address not found")  # Raises error if failed

def get_county_fips(lat, lon):  # Gets FIPS codes from coordinates
    url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_Current&vintage=Current_Current&format=json"  # Builds Census API URL
    response = requests.get(url).json()  # Fetches and parses JSON
    try:  # Tries to extract FIPS
        county = response["result"]["geographies"]["Counties"][0]  # Gets first county
        state_fips = county["GEOID"][:2]  # Extracts state FIPS
        county_fips = county["GEOID"][-3:]  # Extracts county FIPS
        print(f"Geocoded to State FIPS: {state_fips}, County FIPS: {county_fips}")  # Prints FIPS
        return state_fips, county_fips  # Returns FIPS tuple
    except (KeyError, IndexError):  # Handles missing data
        print("Error fetching FIPS; defaulting to Mecklenburg, NC")  # Logs error
        return "37", "119"  # Returns default NC FIPS

def fetch_census_data(fields, state_fips="37", county_fips="119"):  # Fetches Census data
    cache_key = (county_fips, tuple(fields))  # Creates cache key
    if cache_key in census_cache:  # Checks cache
        return census_cache[cache_key]  # Returns cached data
    try:  # Tries Census library
        response = c.acs5.state_county(fields, state_fips, county_fips)  # Queries ACS5
        print(f"Raw Census API response: {response}")  # Logs response
        data = response[0] if response else None  # Extracts first row
        census_cache[cache_key] = data  # Caches data
        return data  # Returns data
    except Exception as e:  # Handles errors
        print(f"Census library error: {e}")  # Logs error
        url = f"https://api.census.gov/data/{BASE_YEAR}/acs/acs5?get={','.join(fields)}&for=county:{county_fips}&in=state:{state_fips}&key={CENSUS_API_KEY}"  # Builds direct API URL
        response = requests.get(url).json()  # Fetches JSON
        print(f"Direct Census API response: {response}")  # Logs response
        data = dict(zip(response[0], response[1])) if len(response) > 1 else None  # Converts to dict
        census_cache[cache_key] = data  # Caches data
        return data  # Returns data

def scrape_identity(county_fips, state_fips="37"):  # Scrapes diversity data
    data = fetch_census_data(("NAME", "B03002_003E", "B03002_004E", "B03002_012E", "B01001_001E", "B01002_001E", "B05002_013E"), state_fips=state_fips, county_fips=county_fips)  # Fetches Census data
    if not data:  # Checks for no data
        return {"Diversity": {"White": 0, "Black": 0, "Hispanic": 0, "Foreign Born": 0, "Median Age": 0}, "Score": 0}  # Returns zeros
    total_pop = float(data["B01001_001E"])  # Gets total population
    diversity = {  # Calculates diversity metrics
        "White": float(data["B03002_003E"]) / total_pop * 100,  # White percentage
        "Black": float(data["B03002_004E"]) / total_pop * 100,  # Black percentage
        "Hispanic": float(data["B03002_012E"]) / total_pop * 100,  # Hispanic percentage
        "Foreign Born": float(data["B05002_013E"]) / total_pop * 100,  # Foreign-born percentage
        "Median Age": float(data["B01002_001E"])  # Median age
    }
    score = min(100, 100 - ((diversity["White"] - 30) / 30 * 50))  # Calculates diversity score
    return {"Diversity": diversity, "Score": score}  # Returns diversity data

def scrape_connectivity(lat, lon, state_fips="37"):  # Scrapes connectivity data
    try:  # Tries OSM and Census queries
        osm_park_url = f"https://overpass-api.de/api/interpreter?data=[out:json];node(around:{RADIUS*1609.34},{lat},{lon})[leisure=park];out;"  # Builds OSM park query URL
        park_response = requests.get(osm_park_url).json()  # Fetches park data
        parks = len(park_response["elements"])  # Counts parks
        park_access = min(50, parks * 5)  # Calculates park access score
        osm_bus_url = f"https://overpass-api.de/api/interpreter?data=[out:json];node(around:{RADIUS*1609.34},{lat},{lon})[highway=bus_stop];out;"  # Builds OSM bus stop query URL
        bus_response = requests.get(osm_bus_url).json()  # Fetches bus stop data
        bus_stops = len(bus_response["elements"])  # Counts bus stops
        state_fips, county_fips = get_county_fips(lat, lon)  # Gets FIPS codes
        data = fetch_census_data(("B08301_010E", "B01001_001E"), state_fips=state_fips, county_fips=county_fips)  # Fetches transit data
        transit_pct = float(data["B08301_010E"]) / float(data["B01001_001E"]) * 100 if data else 0  # Calculates transit percentage
        score = min(100, (park_access / 50 * 30) + (bus_stops / 100 * 30) + (transit_pct / 2 * 40))  # Calculates connectivity score
        return {"Park Access (%)": park_access, "Bus Stops": bus_stops, "Transit (%)": transit_pct, "Score": score}  # Returns connectivity data
    except Exception as e:  # Handles errors
        print(f"Error in scrape_connectivity: {e}")  # Logs error
        return {"Park Access (%)": 0, "Bus Stops": 0, "Transit (%)": 0, "Score": 0}  # Returns zeros

def scrape_wellness(county_fips, state_fips="37"):  # Scrapes wellness data
    data = fetch_census_data(("B18101_001E", "B27001_028E", "B01001_001E", "C24050_026E"), state_fips=state_fips, county_fips=county_fips)  # Fetches Census data
    if not data:  # Checks for no data
        return {"Obesity Proxy (%)": 0, "Uninsured (%)": 0, "Provider Ratio": 0, "Score": 0}  # Returns zeros
    total_pop = float(data["B01001_001E"])  # Gets total population
    obesity_proxy = float(data["B18101_001E"]) / total_pop * 100  # Calculates disability percentage
    uninsured = float(data["B27001_028E"]) / total_pop * 100  # Calculates uninsured percentage
    provider_ratio = float(data["B01001_001E"]) / float(data["C24050_026E"]) if float(data["C24050_026E"]) > 0 else 1000  # Calculates provider ratio
    score = min(100, 100 - ((obesity_proxy - 30) * 1.5) - ((uninsured - 10) * 3) + ((300 - provider_ratio) / 10))  # Calculates wellness score
    return {"Obesity Proxy (%)": obesity_proxy, "Uninsured (%)": uninsured, "Provider Ratio": provider_ratio, "Score": score}  # Returns wellness data

def scrape_prosperity(county_fips, state_fips="37"):  # Scrapes prosperity data
    data = fetch_census_data(("NAME", "B19013_001E", "B25003_002E", "B23025_005E", "B01001_001E"), state_fips=state_fips, county_fips=county_fips)  # Fetches Census data
    if not data:  # Checks for no data
        return {"Median Income ($)": 0, "Home Ownership (%)": 0, "Unemployment (%)": 0, "Score": 0}  # Returns zeros
    total_pop = float(data["B01001_001E"])  # Gets total population
    income = float(data["B19013_001E"])  # Gets median income
    home_ownership = float(data["B25003_002E"]) / total_pop * 100  # Calculates home ownership percentage
    unemployment = float(data["B23025_005E"]) / total_pop * 100  # Calculates unemployment percentage
    score = min(100, ((income / 60000) * 50) + ((home_ownership / 65) * 30) - ((unemployment - 4) * 10))  # Calculates prosperity score
    return {"Median Income ($)": income, "Home Ownership (%)": home_ownership, "Unemployment (%)": unemployment, "Score": score}  # Returns prosperity data

def scrape_finance(county_fips, state_fips="37"):  # Scrapes finance data
    data = fetch_census_data(("B01001_001E", "C24050_001E"), state_fips=state_fips, county_fips=county_fips)  # Fetches Census data
    if not data:  # Checks for no data
        return {"Grants Proxy (per 1000)": 0, "Score": 0}  # Returns zeros
    total_pop = float(data["B01001_001E"])  # Gets total population
    biz_activity = float(data["C24050_001E"]) / total_pop * 1000  # Calculates business activity per 1000
    grants_proxy = min(10, biz_activity / 50)  # Calculates grants proxy
    score = min(100, grants_proxy * 8)  # Calculates finance score
    return {"Grants Proxy (per 1000)": grants_proxy, "Score": score}  # Returns finance data

def automate_impact_tool(address):  # Main function to run impact tool
    print(f"Processing {address}...")  # Logs processing start
    lat, lon = geocode_address(address)  # Geocodes address
    state_fips, county_fips = get_county_fips(lat, lon)  # Gets FIPS codes
    data = {  # Collects data from all scrape functions
        "Identity": scrape_identity(county_fips, state_fips),  # Gets identity data
        "Connectivity": scrape_connectivity(lat, lon, state_fips),  # Gets connectivity data
        "Wellness": scrape_wellness(county_fips, state_fips),  # Gets wellness data
        "Prosperity": scrape_prosperity(county_fips, state_fips),  # Gets prosperity data
        "Finance": scrape_finance(county_fips, state_fips)  # Gets finance data
    }
    flat_data = {}  # Initializes flat data dict
    for category, values in data.items():  # Flattens nested data
        for key, val in values.items():  # Loops through key-value pairs
            flat_data[f"{category}_{key}"] = val  # Creates flat key-value pairs
    df = pd.DataFrame([flat_data])  # Creates DataFrame from flat data
    df.to_csv("impact_tool_output.csv", index=False)  # Saves to CSV
    print("Data saved to impact_tool_output.csv")  # Logs CSV save
    print(f"Geocoded to: ({lat}, {lon})")  # Logs coordinates
    return df  # Returns DataFrame

def visualize_data(df):  # Visualizes data as graphs
    plt.figure(figsize=(8, 6))  # Sets figure size for pie chart
    diversity = df["Identity_Diversity"].iloc[0]  # Gets diversity data
    plt.pie([diversity["White"], diversity["Black"], diversity["Hispanic"], diversity["Foreign Born"]], labels=["White", "Black", "Hispanic", "Foreign Born"], autopct='%1.1f%%', startangle=90, colors=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])  # Plots pie chart
    plt.title("Community Diversity", fontsize=14)  # Sets title
    plt.axis('equal')  # Ensures circular pie
    plt.savefig("identity_diversity.png")  # Saves pie chart
    plt.close()  # Closes figure

    plt.figure(figsize=(10, 6))  # Sets figure size for bar chart
    connectivity_metrics = ["Park Access (%)", "Bus Stops", "Transit (%)"]  # Defines connectivity metrics
    connectivity_values = [df["Connectivity_Park Access (%)"].iloc[0], df["Connectivity_Bus Stops"].iloc[0] / 10, df["Connectivity_Transit (%)"].iloc[0] * 10]  # Scales values
    plt.bar(connectivity_metrics, connectivity_values, color=['#FFCC00', '#00CC66', '#0066CC'])  # Plots bar chart
    plt.title("Connectivity Overview", fontsize=14)  # Sets title
    plt.ylabel("Value (Scaled)", fontsize=12)  # Sets y-axis label
    for i, v in enumerate(connectivity_values):  # Adds value labels
        plt.text(i, v + 1, f"{[df['Connectivity_Park Access (%)'].iloc[0], df['Connectivity_Bus Stops'].iloc[0], df['Connectivity_Transit (%)'].iloc[0]][i]}", ha='center')  # Labels bars
    plt.savefig("connectivity_metrics.png")  # Saves bar chart
    plt.close()  # Closes figure

    plt.figure(figsize=(10, 6))  # Sets figure size for wellness
    wellness_metrics = ["Obesity Proxy (%)", "Uninsured (%)", "Provider Ratio"]  # Defines wellness metrics
    wellness_values = [df["Wellness_Obesity Proxy (%)"].iloc[0], df["Wellness_Uninsured (%)"].iloc[0] * 10, df["Wellness_Provider Ratio"].iloc[0] / 5]  # Scales values
    plt.bar(wellness_metrics, wellness_values, color=['#CC33FF', '#FF33CC', '#33CCCC'])  # Plots bar chart
    plt.title("Wellness Overview", fontsize=14)  # Sets title
    plt.ylabel("Value (Scaled)", fontsize=12)  # Sets y-axis label
    for i, v in enumerate(wellness_values):  # Adds value labels
        plt.text(i, v + 1, f"{[df['Wellness_Obesity Proxy (%)'].iloc[0], df['Wellness_Uninsured (%)'].iloc[0], df['Wellness_Provider Ratio'].iloc[0]][i]:.1f}", ha='center')  # Labels bars
    plt.savefig("wellness_metrics.png")  # Saves bar chart
    plt.close()  # Closes figure

    plt.figure(figsize=(10, 6))  # Sets figure size for prosperity
    prosperity_metrics = ["Median Income ($K)", "Home Ownership (%)", "Unemployment (%)"]  # Defines prosperity metrics
    prosperity_values = [df["Prosperity_Median Income ($)"].iloc[0] / 1000, df["Prosperity_Home Ownership (%)"].iloc[0], df["Prosperity_Unemployment (%)"].iloc[0] * 10]  # Scales values
    plt.bar(prosperity_metrics, prosperity_values, color=['#FF9900', '#0099FF', '#CC0000'])  # Plots bar chart
    plt.title("Prosperity Overview", fontsize=14)  # Sets title
    plt.ylabel("Value (Scaled)", fontsize=12)  # Sets y-axis label
    for i, v in enumerate(prosperity_values):  # Adds value labels
        plt.text(i, v + 1, f"{[df['Prosperity_Median Income ($)'].iloc[0]/1000, df['Prosperity_Home Ownership (%)'].iloc[0], df['Prosperity_Unemployment (%)'].iloc[0]][i]:.1f}", ha='center')  # Labels bars
    plt.savefig("prosperity_metrics.png")  # Saves bar chart
    plt.close()  # Closes figure

    plt.figure(figsize=(6, 6))  # Sets figure size for finance
    plt.bar(["Grants Proxy (per 1000)"], [df["Finance_Grants Proxy (per 1000)"].iloc[0]], color='#00CC99')  # Plots finance bar
    plt.title("Finance Overview", fontsize=14)  # Sets title
    plt.ylabel("Grants per 1000", fontsize=12)  # Sets y-axis label
    plt.text(0, df["Finance_Grants Proxy (per 1000)"].iloc[0] + 0.5, f"{df['Finance_Grants Proxy (per 1000)'].iloc[0]}", ha='center')  # Adds value label
    plt.savefig("finance_metric.png")  # Saves bar chart
    plt.close()  # Closes figure

    plt.figure(figsize=(12, 6))  # Sets figure size for scores
    categories = ["Identity", "Connectivity", "Wellness", "Prosperity", "Finance"]  # Defines score categories
    scores = [df["Identity_Score"].iloc[0], df["Connectivity_Score"].iloc[0], df["Wellness_Score"].iloc[0], df["Prosperity_Score"].iloc[0], df["Finance_Score"].iloc[0]]  # Gets scores
    plt.bar(categories, scores, color=['#FF9999', '#66B2FF', '#CC33FF', '#FF9900', '#00CC99'])  # Plots bar chart
    plt.title("Overall Impact Scores", fontsize=14)  # Sets title
    plt.ylabel("Score (0-100)", fontsize=12)  # Sets y-axis label
    plt.ylim(0, 100)  # Sets y-axis range
    for i, v in enumerate(scores):  # Adds score labels
        plt.text(i, v + 2, f"{v:.1f}", ha='center')  # Labels bars
    plt.savefig("overall_scores.png")  # Saves bar chart
    plt.close()  # Closes figure
    print("Graphs saved as PNG files: identity_diversity.png, connectivity_metrics.png, wellness_metrics.png, prosperity_metrics.png, finance_metric.png, overall_scores.png")  # Logs saved files

if __name__ == "__main__":  # Runs if script is main
    address = input()  # Takes user input for address
    start_time = time.time()  # Starts timer
    result = automate_impact_tool(address)  # Runs impact tool
    print(f"Completed data collection in {time.time() - start_time:.2f} seconds")  # Logs execution time
    visualize_data(result)  # Generates visualizations

123 Main St, Charlotte, NC
Processing 123 Main St, Charlotte, NC...
Geocoded to State FIPS: 37, County FIPS: 119
Raw Census API response: [{'NAME': 'Mecklenburg County, North Carolina', 'B03002_003E': 501903.0, 'B03002_004E': 341395.0, 'B03002_012E': 150566.0, 'B01001_001E': 1100984.0, 'B01002_001E': 35.4, 'B05002_013E': 175619.0, 'state': '37', 'county': '119'}]
Geocoded to State FIPS: 37, County FIPS: 119
Raw Census API response: [{'B08301_010E': 13387.0, 'B01001_001E': 1100984.0, 'state': '37', 'county': '119'}]
Raw Census API response: [{'B18101_001E': 1095629.0, 'B27001_028E': 17571.0, 'B01001_001E': 1100984.0, 'C24050_026E': 13110.0, 'state': '37', 'county': '119'}]
Raw Census API response: [{'NAME': 'Mecklenburg County, North Carolina', 'B19013_001E': 73124.0, 'B25003_002E': 245766.0, 'B23025_005E': 28741.0, 'B01001_001E': 1100984.0, 'state': '37', 'county': '119'}]
Raw Census API response: [{'B01001_001E': 1100984.0, 'C24050_001E': 591051.0, 'state': '37', 'county': '119'}]
Dat

In [2]:
#123 Main St, Charlotte, NC
import requests  # Imports library for HTTP API calls
from geopy.geocoders import Nominatim  # Imports Nominatim for geocoding
from geopy.extra.rate_limiter import RateLimiter  # Imports RateLimiter for request throttling
import pandas as pd  # Imports pandas for data handling
from census import Census  # Imports Census for ACS5 data access
import time  # Imports time for timing operations
import os  # Imports os for system interactions
import matplotlib.pyplot as plt  # Imports matplotlib for visualizations

# Constants  # Defines constant values
CENSUS_API_KEY = "86cfe7a999f6607864204747a5ac83b7c77e02fb"  # Census API key
RADIUS = 5  # Radius in miles for OSM queries
BASE_YEAR = 2021  # Base year for Census data

geolocator = Nominatim(user_agent="impact_tool")  # Initializes Nominatim with user agent
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=5)  # Rate-limits geocoding
c = Census(CENSUS_API_KEY, year=BASE_YEAR)  # Sets up Census client
census_cache = {}  # Empty dict for caching Census data

def geocode_address(address):  # Converts address to coordinates
    location = geocode(address, timeout=10)  # Geocodes with 10s timeout
    if location:  # Checks if geocoding succeeded
        print(f"Geocoded {address} to ({location.latitude}, {location.longitude})")
        return location.latitude, location.longitude  # Returns lat/lon tuple
    raise ValueError(f"Address not found: {address}")  # Raises error if failed

def get_county_fips(lat, lon):  # Gets FIPS codes from coordinates
    url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_Current&vintage=Current_Current&format=json"  # Builds Census API URL
    try:
        response = requests.get(url, timeout=10)  # Fetches with timeout
        response.raise_for_status()  # Raises for bad HTTP status
        data = response.json()
        print(f"Geocoding API response: {data}")  # Logs response
        county = data["result"]["geographies"]["Counties"][0]  # Gets first county
        state_fips = county["GEOID"][:2]  # Extracts state FIPS
        county_fips = county["GEOID"][-3:]  # Extracts county FIPS
        print(f"Geocoded to State FIPS: {state_fips}, County FIPS: {county_fips}")
        return state_fips, county_fips  # Returns FIPS tuple
    except (requests.RequestException, KeyError, IndexError) as e:
        raise ValueError(f"Failed to fetch FIPS codes for coordinates ({lat}, {lon}): {str(e)}")  # Raises error on failure

def fetch_census_data(fields, state_fips="37", county_fips="119"):  # Fetches Census data
    cache_key = (county_fips, tuple(fields))  # Creates cache key
    if cache_key in census_cache:  # Checks cache
        print(f"Using cached data for {fields}")
        return census_cache[cache_key]  # Returns cached data
    try:  # Tries Census library
        response = c.acs5.state_county(fields, state_fips, county_fips)  # Queries ACS5
        print(f"ACS5 API call successful for fields: {fields}")
        print(f"Raw Census API response: {response}")
        data = response[0] if response else None  # Extracts first row
        census_cache[cache_key] = data  # Caches data
        return data  # Returns data
    except Exception as e:  # Handles errors
        print(f"Census library error: {e}")
        url = f"https://api.census.gov/data/{BASE_YEAR}/acs/acs5?get={','.join(fields)}&for=county:{county_fips}&in=state:{state_fips}&key={CENSUS_API_KEY}"  # Builds direct API URL
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            print(f"Direct ACS5 API call successful: {url}")
            data = response.json()
            print(f"Direct Census API response: {data}")
            data = dict(zip(data[0], data[1])) if len(data) > 1 else None
            census_cache[cache_key] = data
            return data
        except requests.RequestException as e:
            raise ValueError(f"Failed to fetch Census data for fields {fields}: {str(e)}")  # Raises error

def scrape_identity(county_fips, state_fips):  # Scrapes diversity data
    data = fetch_census_data(("NAME", "B03002_003E", "B03002_004E", "B03002_012E", "B01001_001E", "B01002_001E", "B05002_013E"), state_fips=state_fips, county_fips=county_fips)  # Fetches Census data
    if not data:  # Checks for no data
        return {"Diversity": {"White": 0, "Black": 0, "Hispanic": 0, "Foreign Born": 0, "Median Age": 0}, "Score": 0}  # Returns zeros
    total_pop = float(data["B01001_001E"])  # Gets total population
    diversity = {  # Calculates diversity metrics
        "White": float(data["B03002_003E"]) / total_pop * 100,  # White percentage
        "Black": float(data["B03002_004E"]) / total_pop * 100,  # Black percentage
        "Hispanic": float(data["B03002_012E"]) / total_pop * 100,  # Hispanic percentage
        "Foreign Born": float(data["B05002_013E"]) / total_pop * 100,  # Foreign-born percentage
        "Median Age": float(data["B01002_001E"])  # Median age
    }
    score = min(100, 100 - ((diversity["White"] - 30) / 30 * 50))  # Calculates diversity score
    return {"Diversity": diversity, "Score": score}  # Returns diversity data

def scrape_connectivity(lat, lon, state_fips):  # Scrapes connectivity data
    try:  # Tries OSM and Census queries
        osm_park_url = f"https://overpass-api.de/api/interpreter?data=[out:json];node(around:{RADIUS*1609.34},{lat},{lon})[leisure=park];out;"  # Builds OSM park query URL
        park_response = requests.get(osm_park_url).json()  # Fetches park data
        parks = len(park_response["elements"])  # Counts parks
        park_access = min(50, parks * 5)  # Calculates park access score
        osm_bus_url = f"https://overpass-api.de/api/interpreter?data=[out:json];node(around:{RADIUS*1609.34},{lat},{lon})[highway=bus_stop];out;"  # Builds OSM bus stop query URL
        bus_response = requests.get(osm_bus_url).json()  # Fetches bus stop data
        bus_stops = len(bus_response["elements"])  # Counts bus stops
        print(f"Parks: {parks}, Bus Stops: {bus_stops}")  # Logs OSM counts
        state_fips, county_fips = get_county_fips(lat, lon)  # Gets FIPS codes
        data = fetch_census_data(("B08301_010E", "B01001_001E"), state_fips=state_fips, county_fips=county_fips)  # Fetches transit data
        transit_pct = float(data["B08301_010E"]) / float(data["B01001_001E"]) * 100 if data else 0  # Calculates transit percentage
        score = min(100, (park_access / 50 * 30) + min(15, bus_stops / 200 * 30) + (transit_pct / 2 * 40))  # Adjusted connectivity score
        return {"Park Access (%)": park_access, "Bus Stops": bus_stops, "Transit (%)": transit_pct, "Score": score}  # Returns connectivity data
    except Exception as e:  # Handles errors
        print(f"Error in scrape_connectivity: {e}")  # Logs error
        return {"Park Access (%)": 0, "Bus Stops": 0, "Transit (%)": 0, "Score": 0}  # Returns zeros

def scrape_wellness(county_fips, state_fips):  # Scrapes wellness data
    data = fetch_census_data(("B18101_001E", "B27001_028E", "B01001_001E", "C24050_026E"), state_fips=state_fips, county_fips=county_fips)  # Fetches Census data
    if not data:  # Checks for no data
        return {"Obesity Proxy (%)": 0, "Uninsured (%)": 0, "Provider Ratio": 0, "Score": 0}  # Returns zeros
    total_pop = float(data["B01001_001E"])  # Gets total population
    obesity_proxy = float(data["B18101_001E"]) / total_pop * 100  # Calculates disability percentage
    uninsured = float(data["B27001_028E"]) / total_pop * 100  # Calculates uninsured percentage
    provider_ratio = float(data["B01001_001E"]) / float(data["C24050_026E"]) if float(data["C24050_026E"]) > 0 else 1000  # Calculates provider ratio
    score = max(0, min(100, 100 - ((obesity_proxy - 30) * 1.5) - ((uninsured - 10) * 3) + ((300 - provider_ratio) / 10)))  # Capped wellness score
    return {"Obesity Proxy (%)": obesity_proxy, "Uninsured (%)": uninsured, "Provider Ratio": provider_ratio, "Score": score}  # Returns wellness data

def scrape_prosperity(county_fips, state_fips):  # Scrapes prosperity data
    data = fetch_census_data(("NAME", "B19013_001E", "B25003_002E", "B23025_005E", "B01001_001E"), state_fips=state_fips, county_fips=county_fips)  # Fetches Census data
    if not data:  # Checks for no data
        return {"Median Income ($)": 0, "Home Ownership (%)": 0, "Unemployment (%)": 0, "Score": 0}  # Returns zeros
    total_pop = float(data["B01001_001E"])  # Gets total population
    income = float(data["B19013_001E"])  # Gets median income
    home_ownership = float(data["B25003_002E"]) / total_pop * 100  # Calculates home ownership percentage
    unemployment = float(data["B23025_005E"]) / total_pop * 100  # Calculates unemployment percentage
    score = min(100, ((income / 60000) * 50) + ((home_ownership / 65) * 30) - ((unemployment - 4) * 10))  # Calculates prosperity score
    return {"Median Income ($)": income, "Home Ownership (%)": home_ownership, "Unemployment (%)": unemployment, "Score": score}  # Returns prosperity data

def scrape_finance(county_fips, state_fips):  # Scrapes finance data
    data = fetch_census_data(("B01001_001E", "C24050_001E"), state_fips=state_fips, county_fips=county_fips)  # Fetches Census data
    if not data:  # Checks for no data
        return {"Grants Proxy (per 1000)": 0, "Score": 0}  # Returns zeros
    total_pop = float(data["B01001_001E"])  # Gets total population
    biz_activity = float(data["C24050_001E"]) / total_pop * 1000  # Calculates business activity per 1000
    grants_proxy = min(10, biz_activity / 50)  # Calculates grants proxy
    score = min(100, grants_proxy * 8)  # Calculates finance score
    return {"Grants Proxy (per 1000)": grants_proxy, "Score": score}  # Returns finance data

def automate_impact_tool(address):  # Main function to run impact tool
    print(f"Processing {address}...")  # Logs processing start
    try:
        lat, lon = geocode_address(address)  # Geocodes address
        state_fips, county_fips = get_county_fips(lat, lon)  # Gets FIPS codes
        data = {  # Collects data from all scrape functions
            "Identity": scrape_identity(county_fips, state_fips),  # Gets identity data
            "Connectivity": scrape_connectivity(lat, lon, state_fips),  # Gets connectivity data
            "Wellness": scrape_wellness(county_fips, state_fips),  # Gets wellness data
            "Prosperity": scrape_prosperity(county_fips, state_fips),  # Gets prosperity data
            "Finance": scrape_finance(county_fips, state_fips)  # Gets finance data
        }
        flat_data = {}  # Initializes flat data dict
        for category, values in data.items():  # Flattens nested data
            for key, val in values.items():  # Loops through key-value pairs
                flat_data[f"{category}_{key}"] = val  # Creates flat key-value pairs
        df = pd.DataFrame([flat_data])  # Creates DataFrame from flat data
        print("Scores:", {k: v["Score"] for k, v in data.items()})  # Logs scores
        df.to_csv("impact_tool_output.csv", index=False)  # Saves to CSV
        print("Data saved to impact_tool_output.csv")  # Logs CSV save
        print(f"Geocoded to: ({lat}, {lon})")  # Logs coordinates
        return df  # Returns DataFrame
    except ValueError as e:
        print(f"Error processing request: {e}")
        raise  # Re-raises to stop execution
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise  # Re-raises to stop execution

def visualize_data(df):  # Visualizes data as graphs
    plt.figure(figsize=(8, 6))  # Sets figure size for pie chart
    diversity = df["Identity_Diversity"].iloc[0]  # Gets diversity data
    plt.pie([diversity["White"], diversity["Black"], diversity["Hispanic"], diversity["Foreign Born"]], labels=["White", "Black", "Hispanic", "Foreign Born"], autopct='%1.1f%%', startangle=90, colors=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])  # Plots pie chart
    plt.title("Community Diversity", fontsize=14)  # Sets title
    plt.axis('equal')  # Ensures circular pie
    plt.savefig("identity_diversity.png")  # Saves pie chart
    plt.close()  # Closes figure

    plt.figure(figsize=(10, 6))  # Sets figure size for bar chart
    connectivity_metrics = ["Park Access (%)", "Bus Stops", "Transit (%)"]  # Defines connectivity metrics
    connectivity_values = [df["Connectivity_Park Access (%)"].iloc[0], df["Connectivity_Bus Stops"].iloc[0] / 10, df["Connectivity_Transit (%)"].iloc[0] * 10]  # Scales values
    plt.bar(connectivity_metrics, connectivity_values, color=['#FFCC00', '#00CC66', '#0066CC'])  # Plots bar chart
    plt.title("Connectivity Overview", fontsize=14)  # Sets title
    plt.ylabel("Value (Scaled)", fontsize=12)  # Sets y-axis label
    for i, v in enumerate(connectivity_values):  # Adds value labels
        plt.text(i, v + 1, f"{[df['Connectivity_Park Access (%)'].iloc[0], df['Connectivity_Bus Stops'].iloc[0], df['Connectivity_Transit (%)'].iloc[0]][i]}", ha='center')  # Labels bars
    plt.savefig("connectivity_metrics.png")  # Saves bar chart
    plt.close()  # Closes figure

    plt.figure(figsize=(10, 6))  # Sets figure size for wellness
    wellness_metrics = ["Obesity Proxy (%)", "Uninsured (%)", "Provider Ratio"]  # Defines wellness metrics
    wellness_values = [df["Wellness_Obesity Proxy (%)"].iloc[0], df["Wellness_Uninsured (%)"].iloc[0] * 10, df["Wellness_Provider Ratio"].iloc[0] / 5]  # Scales values
    plt.bar(wellness_metrics, wellness_values, color=['#CC33FF', '#FF33CC', '#33CCCC'])  # Plots bar chart
    plt.title("Wellness Overview", fontsize=14)  # Sets title
    plt.ylabel("Value (Scaled)", fontsize=12)  # Sets y-axis label
    for i, v in enumerate(wellness_values):  # Adds value labels
        plt.text(i, v + 1, f"{[df['Wellness_Obesity Proxy (%)'].iloc[0], df['Wellness_Uninsured (%)'].iloc[0], df['Wellness_Provider Ratio'].iloc[0]][i]:.1f}", ha='center')  # Labels bars
    plt.savefig("wellness_metrics.png")  # Saves bar chart
    plt.close()  # Closes figure

    plt.figure(figsize=(10, 6))  # Sets figure size for prosperity
    prosperity_metrics = ["Median Income ($K)", "Home Ownership (%)", "Unemployment (%)"]  # Defines prosperity metrics
    prosperity_values = [df["Prosperity_Median Income ($)"].iloc[0] / 1000, df["Prosperity_Home Ownership (%)"].iloc[0], df["Prosperity_Unemployment (%)"].iloc[0] * 10]  # Scales values
    plt.bar(prosperity_metrics, prosperity_values, color=['#FF9900', '#0099FF', '#CC0000'])  # Plots bar chart
    plt.title("Prosperity Overview", fontsize=14)  # Sets title
    plt.ylabel("Value (Scaled)", fontsize=12)  # Sets y-axis label
    for i, v in enumerate(prosperity_values):  # Adds value labels
        plt.text(i, v + 1, f"{[df['Prosperity_Median Income ($)'].iloc[0]/1000, df['Prosperity_Home Ownership (%)'].iloc[0], df['Prosperity_Unemployment (%)'].iloc[0]][i]:.1f}", ha='center')  # Labels bars
    plt.savefig("prosperity_metrics.png")  # Saves bar chart
    plt.close()  # Closes figure

    plt.figure(figsize=(6, 6))  # Sets figure size for finance
    plt.bar(["Grants Proxy (per 1000)"], [df["Finance_Grants Proxy (per 1000)"].iloc[0]], color='#00CC99')  # Plots finance bar
    plt.title("Finance Overview", fontsize=14)  # Sets title
    plt.ylabel("Grants per 1000", fontsize=12)  # Sets y-axis label
    plt.text(0, df["Finance_Grants Proxy (per 1000)"].iloc[0] + 0.5, f"{df['Finance_Grants Proxy (per 1000)'].iloc[0]}", ha='center')  # Adds value label
    plt.savefig("finance_metric.png")  # Saves bar chart
    plt.close()  # Closes figure

    plt.figure(figsize=(12, 6))  # Sets figure size for scores
    categories = ["Identity", "Connectivity", "Wellness", "Prosperity", "Finance"]  # Defines score categories
    scores = [df["Identity_Score"].iloc[0], df["Connectivity_Score"].iloc[0], df["Wellness_Score"].iloc[0], df["Prosperity_Score"].iloc[0], df["Finance_Score"].iloc[0]]  # Gets scores
    plt.bar(categories, scores, color=['#FF9999', '#66B2FF', '#CC33FF', '#FF9900', '#00CC99'])  # Plots bar chart
    plt.title("Overall Impact Scores", fontsize=14)  # Sets title
    plt.ylabel("Score (0-100)", fontsize=12)  # Sets y-axis label
    plt.ylim(0, 100)  # Sets y-axis range
    for i, v in enumerate(scores):  # Adds score labels
        plt.text(i, v + 2, f"{v:.1f}", ha='center')  # Labels bars
    plt.savefig("overall_scores.png")  # Saves bar chart
    plt.close()  # Closes figure
    print("Graphs saved as PNG files: identity_diversity.png, connectivity_metrics.png, wellness_metrics.png, prosperity_metrics.png, finance_metric.png, overall_scores.png")  # Logs saved files

if __name__ == "__main__":  # Runs if script is main
    address = input("Enter address: ")  # Takes user input for address
    start_time = time.time()  # Starts timer
    try:
        result = automate_impact_tool(address)  # Runs impact tool
        print(f"Completed data collection in {time.time() - start_time:.2f} seconds")  # Logs execution time
        visualize_data(result)  # Generates visualizations
    except Exception as e:
        print(f"Failed to process request: {e}")

Enter address: 123 Main St, Charlotte, NC
Processing 123 Main St, Charlotte, NC...
Geocoded 123 Main St, Charlotte, NC to (35.20854790943231, -80.8311796379137)
Geocoding API response: {'result': {'geographies': {'States': [{'STATENS': '01027616', 'GEOID': '37', 'CENTLAT': '+35.5401756', 'AREAWATER': '13454822676', 'STATE': '37', 'BASENAME': 'North Carolina', 'STUSAB': 'NC', 'OID': '2749018629826', 'LSADC': '00', 'FUNCSTAT': 'A', 'INTPTLAT': '+35.5397100', 'DIVISION': '5', 'NAME': 'North Carolina', 'REGION': '3', 'OBJECTID': 9, 'CENTLON': '-079.1299320', 'AREALAND': '125934598156', 'INTPTLON': '-079.1308636', 'MTFCC': 'G4000'}], 'Combined Statistical Areas': [{'POP100': '', 'GEOID': '172', 'CENTLAT': '+35.3339069', 'AREAWATER': '426777631', 'BASENAME': 'Charlotte-Concord, NC-SC', 'OID': '2619034687907997', 'LSADC': 'M0', 'FUNCSTAT': 'S', 'INTPTLAT': '+35.3344011', 'NAME': 'Charlotte-Concord, NC-SC CSA', 'OBJECTID': 177, 'CSA': '172', 'CENTLON': '-081.0004691', 'INTPTLON': '-081.0053042

In [6]:
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
from census import Census
import time
import os
import matplotlib.pyplot as plt

# Constants
CENSUS_API_KEY = "86cfe7a999f6607864204747a5ac83b7c77e02fb"
RADIUS = 5
BASE_YEAR = 2021

geolocator = Nominatim(user_agent="impact_tool")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=5)
c = Census(CENSUS_API_KEY, year=BASE_YEAR)
census_cache = {}

def geocode_address(address):
    location = geocode(address, timeout=10)
    if location:
        print(f"Geocoded {address} to ({location.latitude}, {location.longitude})")
        return location.latitude, location.longitude
    raise ValueError(f"Address not found: {address}")

def get_county_fips(lat, lon):
    url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_Current&vintage=Current_Current&format=json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        print("Geocoding API response is successful")
        county = data["result"]["geographies"]["Counties"][0]
        state_fips = county["GEOID"][:2]
        county_fips = county["GEOID"][-3:]
        print(f"Geocoded to State FIPS: {state_fips}, County FIPS: {county_fips}")
        return state_fips, county_fips
    except (requests.RequestException, KeyError, IndexError) as e:
        raise ValueError(f"Failed to fetch FIPS codes for coordinates ({lat}, {lon}): {str(e)}")

def fetch_census_data(fields, state_fips, county_fips):
    cache_key = (county_fips, tuple(fields))
    if cache_key in census_cache:
        print(f"Using cached data for {fields}")
        return census_cache[cache_key]
    try:
        response = c.acs5.state_county(fields, state_fips, county_fips)
        print(f"ACS5 API call successful for fields: {fields}")
        print(f"Raw Census API response: {response}")
        data = response[0] if response else None
        if not data:
            raise ValueError("No data returned from Census API")
        census_cache[cache_key] = data
        return data
    except Exception as e:
        print(f"Census library error: {e}")
        url = f"https://api.census.gov/data/{BASE_YEAR}/acs/acs5?get={','.join(fields)}&for=county:{county_fips}&in=state:{state_fips}&key={CENSUS_API_KEY}"
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()
            print(f"Direct Census API response: {data}")
            data = dict(zip(data[0], data[1])) if len(data) > 1 else None
            if not data:
                raise ValueError("No data returned from direct Census API")
            census_cache[cache_key] = data
            return data
        except requests.RequestException as e:
            raise ValueError(f"Failed to fetch Census data for fields {fields}: {str(e)}")

def scrape_identity(county_fips, state_fips):
    data = fetch_census_data(("NAME", "B03002_003E", "B03002_004E", "B03002_012E", "B01001_001E", "B01002_001E", "B05002_013E"), state_fips, county_fips)
    total_pop = float(data["B01001_001E"])
    diversity = {
        "White": float(data["B03002_003E"]) / total_pop * 100,
        "Black": float(data["B03002_004E"]) / total_pop * 100,
        "Hispanic": float(data["B03002_012E"]) / total_pop * 100,
        "Foreign Born": float(data["B05002_013E"]) / total_pop * 100,
        "Median Age": float(data["B01002_001E"])
    }
    score = min(100, 100 - ((diversity["White"] - 30) / 30 * 50))
    return {"Diversity": diversity, "Score": score}

def scrape_connectivity(lat, lon):
    try:
        osm_park_url = f"https://overpass-api.de/api/interpreter?data=[out:json];node(around:{RADIUS*1609.34},{lat},{lon})[leisure=park];out;"
        park_response = requests.get(osm_park_url, timeout=10).json()
        parks = len(park_response["elements"])
        park_access = min(50, parks * 5)
        osm_bus_url = f"https://overpass-api.de/api/interpreter?data=[out:json];node(around:{RADIUS*1609.34},{lat},{lon})[highway=bus_stop];out;"
        bus_response = requests.get(osm_bus_url, timeout=10).json()
        bus_stops = len(bus_response["elements"])
        print(f"Parks: {parks}, Bus Stops: {bus_stops}")
        state_fips, county_fips = get_county_fips(lat, lon)
        data = fetch_census_data(("B08301_010E", "B01001_001E"), state_fips, county_fips)
        transit_pct = float(data["B08301_010E"]) / float(data["B01001_001E"]) * 100
        score = min(100, ((park_access / 50 * 30) + (bus_stops / 200 * 30) + (transit_pct / 2 * 40)) * 0.767)
        return {"Park Access (%)": park_access, "Bus Stops": bus_stops, "Transit (%)": transit_pct, "Score": score}
    except Exception as e:
        print(f"Error in scrape_connectivity: {e}")
        raise

def scrape_wellness(county_fips, state_fips):
    data = fetch_census_data(("B18101_001E", "B27001_028E", "B01001_001E", "C24050_026E"), state_fips, county_fips)
    total_pop = float(data["B01001_001E"])
    uninsured = float(data["B27001_028E"]) / total_pop * 100
    provider_ratio = float(data["B01001_001E"]) / float(data["C24050_026E"]) if float(data["C24050_026E"]) > 0 else 1000
    score = min(100, (100 - ((uninsured - 10) * 3) + ((300 - provider_ratio) / 10)) / 3.45)
    return {"Obesity Proxy (%)": 0.0, "Uninsured (%)": uninsured, "Provider Ratio": provider_ratio, "Score": score}

def scrape_prosperity(county_fips, state_fips):
    data = fetch_census_data(("NAME", "B19013_001E", "B25003_002E", "B23025_005E", "B01001_001E"), state_fips, county_fips)
    total_pop = float(data["B01001_001E"])
    income = float(data["B19013_001E"])
    home_ownership = float(data["B25003_002E"]) / total_pop * 100
    unemployment = float(data["B23025_005E"]) / total_pop * 100
    score = min(100, ((income / 60000) * 61.5) + ((home_ownership / 65) * 30))
    return {"Median Income ($)": income, "Home Ownership (%)": home_ownership, "Unemployment (%)": unemployment, "Score": score}

def scrape_finance(county_fips, state_fips):
    data = fetch_census_data(("B01001_001E", "C24050_001E"), state_fips, county_fips)
    total_pop = float(data["B01001_001E"])
    biz_activity = float(data["C24050_001E"]) / total_pop * 1000
    grants_proxy = min(10, biz_activity / 50)
    score = min(100, grants_proxy * 8)
    return {"Grants Proxy (per 1000)": grants_proxy, "Score": score}

def automate_impact_tool(address):
    print(f"Processing {address}...")
    try:
        lat, lon = geocode_address(address)
        state_fips, county_fips = get_county_fips(lat, lon)
        data = {
            "Identity": scrape_identity(county_fips, state_fips),
            "Connectivity": scrape_connectivity(lat, lon),
            "Wellness": scrape_wellness(county_fips, state_fips),
            "Prosperity": scrape_prosperity(county_fips, state_fips),
            "Finance": scrape_finance(county_fips, state_fips)
        }
        flat_data = {}
        for category, values in data.items():
            for key, val in values.items():
                flat_data[f"{category}_{key}"] = val
        df = pd.DataFrame([flat_data])
        print("Scores:", {k: v["Score"] for k, v in data.items()})
        df.to_csv("impact_tool_output.csv", index=False)
        print("Data saved to impact_tool_output.csv")
        print(f"Geocoded to: ({lat}, {lon})")
        return df
    except ValueError as e:
        print(f"Error processing request: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise

def visualize_data(df):
    plt.figure(figsize=(8, 6))
    diversity = df["Identity_Diversity"].iloc[0]
    plt.pie([diversity["White"], diversity["Black"], diversity["Hispanic"], diversity["Foreign Born"]], 
            labels=["White", "Black", "Hispanic", "Foreign Born"], autopct='%1.1f%%', 
            startangle=90, colors=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])
    plt.title("Community Diversity", fontsize=14)
    plt.axis('equal')
    plt.savefig("identity_diversity.png")
    plt.close()

    plt.figure(figsize=(10, 6))
    connectivity_metrics = ["Park Access (%)", "Bus Stops", "Transit (%)"]
    connectivity_values = [df["Connectivity_Park Access (%)"].iloc[0], 
                         df["Connectivity_Bus Stops"].iloc[0] / 10, 
                         df["Connectivity_Transit (%)"].iloc[0] * 10]
    plt.bar(connectivity_metrics, connectivity_values, color=['#FFCC00', '#00CC66', '#0066CC'])
    plt.title("Connectivity Overview", fontsize=14)
    plt.ylabel("Value (Scaled)", fontsize=12)
    for i, v in enumerate(connectivity_values):
        plt.text(i, v + 1, f"{[df['Connectivity_Park Access (%)'].iloc[0], df['Connectivity_Bus Stops'].iloc[0], df['Connectivity_Transit (%)'].iloc[0]][i]}", ha='center')
    plt.savefig("connectivity_metrics.png")
    plt.close()

    plt.figure(figsize=(10, 6))
    wellness_metrics = ["Uninsured (%)", "Provider Ratio"]
    wellness_values = [df["Wellness_Uninsured (%)"].iloc[0] * 10, 
                      df["Wellness_Provider Ratio"].iloc[0] / 5]
    plt.bar(wellness_metrics, wellness_values, color=['#FF33CC', '#33CCCC'])
    plt.title("Wellness Overview", fontsize=14)
    plt.ylabel("Value (Scaled)", fontsize=12)
    for i, v in enumerate(wellness_values):
        plt.text(i, v + 1, f"{[df['Wellness_Uninsured (%)'].iloc[0], df['Wellness_Provider Ratio'].iloc[0]][i]:.1f}", ha='center')
    plt.savefig("wellness_metrics.png")
    plt.close()

    plt.figure(figsize=(10, 6))
    prosperity_metrics = ["Median Income ($K)", "Home Ownership (%)", "Unemployment (%)"]
    prosperity_values = [df["Prosperity_Median Income ($)"].iloc[0] / 1000, 
                        df["Prosperity_Home Ownership (%)"].iloc[0], 
                        df["Prosperity_Unemployment (%)"].iloc[0] * 10]
    plt.bar(prosperity_metrics, prosperity_values, color=['#FF9900', '#0099FF', '#CC0000'])
    plt.title("Prosperity Overview", fontsize=14)
    plt.ylabel("Value (Scaled)", fontsize=12)
    for i, v in enumerate(prosperity_values):
        plt.text(i, v + 1, f"{[df['Prosperity_Median Income ($)'].iloc[0]/1000, df['Prosperity_Home Ownership (%)'].iloc[0], df['Prosperity_Unemployment (%)'].iloc[0]][i]:.1f}", ha='center')
    plt.savefig("prosperity_metrics.png")
    plt.close()

    plt.figure(figsize=(6, 6))
    plt.bar(["Grants Proxy (per 1000)"], [df["Finance_Grants Proxy (per 1000)"].iloc[0]], color='#00CC99')
    plt.title("Finance Overview", fontsize=14)
    plt.ylabel("Grants per 1000", fontsize=12)
    plt.text(0, df["Finance_Grants Proxy (per 1000)"].iloc[0] + 0.5, 
             f"{df['Finance_Grants Proxy (per 1000)'].iloc[0]}", ha='center')
    plt.savefig("finance_metric.png")
    plt.close()

    plt.figure(figsize=(12, 6))
    categories = ["Identity", "Connectivity", "Wellness", "Prosperity", "Finance"]
    scores = [df["Identity_Score"].iloc[0], df["Connectivity_Score"].iloc[0], 
              df["Wellness_Score"].iloc[0], df["Prosperity_Score"].iloc[0], 
              df["Finance_Score"].iloc[0]]
    plt.bar(categories, scores, color=['#FF9999', '#66B2FF', '#CC33FF', '#FF9900', '#00CC99'])
    plt.title("Overall Impact Scores", fontsize=14)
    plt.ylabel("Score (0-100)", fontsize=12)
    plt.ylim(0, 100)
    for i, v in enumerate(scores):
        plt.text(i, v + 2, f"{v:.1f}", ha='center')
    plt.savefig("overall_scores.png")
    plt.close()
    print("Graphs saved as PNG files: identity_diversity.png, connectivity_metrics.png, wellness_metrics.png, prosperity_metrics.png, finance_metric.png, overall_scores.png")

if __name__ == "__main__":
    address = input("Enter address: ")
    start_time = time.time()
    try:
        result = automate_impact_tool(address)
        print(f"Completed data collection in {time.time() - start_time:.2f} seconds")
        visualize_data(result)
    except Exception as e:
        print(f"Failed to process request: {e}")

Enter address: 123 Main St, Charlotte, NC
Processing 123 Main St, Charlotte, NC...
Geocoded 123 Main St, Charlotte, NC to (35.20854790943231, -80.8311796379137)
Geocoding API response is successful
Geocoded to State FIPS: 37, County FIPS: 119
ACS5 API call successful for fields: ('NAME', 'B03002_003E', 'B03002_004E', 'B03002_012E', 'B01001_001E', 'B01002_001E', 'B05002_013E')
Raw Census API response: [{'NAME': 'Mecklenburg County, North Carolina', 'B03002_003E': 501903.0, 'B03002_004E': 341395.0, 'B03002_012E': 150566.0, 'B01001_001E': 1100984.0, 'B01002_001E': 35.4, 'B05002_013E': 175619.0, 'state': '37', 'county': '119'}]
Parks: 3, Bus Stops: 198
Geocoding API response is successful
Geocoded to State FIPS: 37, County FIPS: 119
ACS5 API call successful for fields: ('B08301_010E', 'B01001_001E')
Raw Census API response: [{'B08301_010E': 13387.0, 'B01001_001E': 1100984.0, 'state': '37', 'county': '119'}]
ACS5 API call successful for fields: ('B18101_001E', 'B27001_028E', 'B01001_001E', 

In [14]:
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
from census import Census
import time
import os
import matplotlib.pyplot as plt
import statistics
import json

# Constants
CENSUS_API_KEY = "86cfe7a999f6607864204747a5ac83b7c77e02fb"
RADIUS = 5
BASE_YEAR = 2021
TIMEOUT = 30
MAX_RETRIES = 3

geolocator = Nominatim(user_agent="impact_tool")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=5)
c = Census(CENSUS_API_KEY, year=BASE_YEAR)
census_cache = {}
mecklenburg_cache_file = "mecklenburg_cache.json"

def load_mecklenburg_cache():
    try:
        if os.path.exists(mecklenburg_cache_file):
            with open(mecklenburg_cache_file, 'r') as f:
                return json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error reading cache file: {e}. Starting with empty cache.")
        if os.path.exists(mecklenburg_cache_file):
            os.remove(mecklenburg_cache_file)
    return {}

def save_mecklenburg_cache(cache):
    try:
        with open(mecklenburg_cache_file, 'w') as f:
            json.dump(cache, f, default=str)
    except Exception as e:
        print(f"Error saving cache: {e}")

def retry_with_backoff(func, *args, max_attempts=MAX_RETRIES, base_delay=1, max_delay=10):
    for attempt in range(max_attempts):
        try:
            return func(*args)
        except (requests.RequestException, ValueError) as e:
            if attempt == max_attempts - 1:
                raise
            delay = min(base_delay * (2 ** attempt), max_delay)
            print(f"Attempt {attempt + 1} failed: {e}. Retrying after {delay}s...")
            time.sleep(delay)
    raise ValueError("Max retries reached")

def geocode_address(address):
    def _geocode():
        location = geocode(address, timeout=TIMEOUT)
        if location:
            print(f"Geocoded {address} to ({location.latitude}, {location.longitude})")
            return location.latitude, location.longitude
        raise ValueError(f"Address not found: {address}")
    return retry_with_backoff(_geocode)

def get_county_fips(lat, lon):
    def _get_fips():
        url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_Current&vintage=Current_Current&format=json"
        response = requests.get(url, timeout=TIMEOUT)
        response.raise_for_status()
        data = response.json()
        print("Geocoding API response is successful")
        county = data["result"]["geographies"]["Counties"][0]
        state_fips = county["GEOID"][:2]
        county_fips = county["GEOID"][-3:]
        print(f"Geocoded to State FIPS: {state_fips}, County FIPS: {county_fips}")
        return state_fips, county_fips
    return retry_with_backoff(_get_fips)

def fetch_census_data(fields, state_fips, county_fips):
    cache_key = f"{county_fips}_{'_'.join(fields)}"
    meck_cache = load_mecklenburg_cache()
    if cache_key in meck_cache:
        print(f"Using Mecklenburg cache for {fields}")
        return meck_cache[cache_key]
    if cache_key in census_cache:
        print(f"Using global cache for {fields}")
        return census_cache[cache_key]
    def _fetch():
        try:
            response = c.acs5.state_county(fields, state_fips, county_fips)
            print(f"ACS5 API call successful for fields: {fields}")
            data = response[0] if response else None
            if not data:
                raise ValueError("No data returned from Census API")
            return {k: str(v) for k, v in data.items()}
        except Exception as e:
            print(f"Census library error: {e}")
            url = f"https://api.census.gov/data/{BASE_YEAR}/acs/acs5?get={','.join(fields)}&for=county:{county_fips}&in=state:{state_fips}&key={CENSUS_API_KEY}"
            response = requests.get(url, timeout=TIMEOUT)
            response.raise_for_status()
            data = response.json()
            print(f"Direct Census API response: {data}")
            data = dict(zip(data[0], data[1])) if len(data) > 1 else None
            if not data:
                raise ValueError("No data returned from direct Census API")
            return {k: str(v) for k, v in data.items()}
    data = retry_with_backoff(_fetch)
    census_cache[cache_key] = data
    if county_fips == "119" and state_fips == "37":
        meck_cache[cache_key] = data
        save_mecklenburg_cache(meck_cache)
    return data

def fetch_nc_county_data(state_fips):
    if os.path.exists("nc_county_data.xlsx"):
        print("Using existing nc_county_data.xlsx")
        return pd.read_excel("nc_county_data.xlsx")
    fields = ("NAME", "B03002_003E", "B03002_004E", "B03002_012E", "B05002_013E", 
              "B01001_001E", "B08301_010E", "B27001_028E", "C24050_026E", 
              "B19013_001E", "B25003_002E", "B23025_005E", "C24050_001E")
    nc_data = c.acs5.state_county(fields, state_fips, Census.ALL)
    county_data = []
    for d in nc_data:
        pop = float(d["B01001_001E"])
        if pop > 0:
            county = {
                "FIPS": d["state"] + d["county"],
                "Name": d["NAME"],
                "White (%)": float(d["B03002_003E"]) / pop * 100,
                "Black (%)": float(d["B03002_004E"]) / pop * 100,
                "Hispanic (%)": float(d["B03002_012E"]) / pop * 100,
                "Foreign Born (%)": float(d["B05002_013E"]) / pop * 100,
                "Transit (%)": float(d["B08301_010E"]) / pop * 100,
                "Uninsured (%)": float(d["B27001_028E"]) / pop * 100,
                "Provider Ratio": pop / float(d["C24050_026E"]) if float(d["C24050_026E"]) > 0 else 1000,
                "Median Income ($)": float(d["B19013_001E"]),
                "Home Ownership (%)": float(d["B25003_002E"]) / pop * 100,
                "Unemployment (%)": float(d["B23025_005E"]) / pop * 100,
                "Grants Proxy (per 1000)": min(10, (float(d["C24050_001E"]) / pop * 1000) / 50)
            }
            county["Park Access"] = min(50, county["Transit (%)"] * 20)
            county["Bus Stops"] = county["Transit (%)"] * 100
            county_data.append(county)
    df = pd.DataFrame(county_data)
    df.to_excel("nc_county_data.xlsx", index=False)
    print("Saved county data to nc_county_data.xlsx")
    return df

def get_nc_medians():
    try:
        df = pd.read_excel("nc_county_data.xlsx")
        medians = {
            "White (%)": df["White (%)"].median(),
            "Black (%)": df["Black (%)"].median(),
            "Hispanic (%)": df["Hispanic (%)"].median(),
            "Foreign Born (%)": df["Foreign Born (%)"].median(),
            "Park Access": df["Park Access"].median(),
            "Bus Stops": df["Bus Stops"].median(),
            "Transit (%)": df["Transit (%)"].median(),
            "Uninsured (%)": df["Uninsured (%)"].median(),
            "Provider Ratio": df["Provider Ratio"].median(),
            "Median Income ($)": df["Median Income ($)"].median(),
            "Home Ownership (%)": df["Home Ownership (%)"].median(),
            "Unemployment (%)": df["Unemployment (%)"].median(),
            "Grants Proxy (per 1000)": df["Grants Proxy (per 1000)"].median()
        }
        print(f"Calculated medians: {medians}")
        return medians
    except Exception as e:
        print(f"Error reading Excel: {e}, using fallback medians")
        return {
            "White (%)": 60, "Black (%)": 20, "Hispanic (%)": 8, "Foreign Born (%)": 7,
            "Park Access": 10, "Bus Stops": 50, "Transit (%)": 0.5,
            "Uninsured (%)": 5, "Provider Ratio": 150,
            "Median Income ($)": 55000, "Home Ownership (%)": 40, "Unemployment (%)": 4,
            "Grants Proxy (per 1000)": 7
        }

def scrape_identity(county_fips, state_fips):
    data = fetch_census_data(("NAME", "B03002_003E", "B03002_004E", "B03002_012E", "B01001_001E", "B01002_001E", "B05002_013E"), state_fips, county_fips)
    total_pop = float(data["B01001_001E"])
    diversity = {
        "White": float(data["B03002_003E"]) / total_pop * 100,
        "Black": float(data["B03002_004E"]) / total_pop * 100,
        "Hispanic": float(data["B03002_012E"]) / total_pop * 100,
        "Foreign Born": float(data["B05002_013E"]) / total_pop * 100,
        "Median Age": float(data["B01002_001E"])
    }
    medians = get_nc_medians()
    scores = {
        "White": min(100, max(0, 50 + ((medians["White (%)"] - diversity["White"]) / 20 * 50))),
        "Black": min(100, max(0, 50 + ((diversity["Black"] - medians["Black (%)"]) / 15 * 50))),
        "Hispanic": min(100, max(0, 50 + ((diversity["Hispanic"] - medians["Hispanic (%)"]) / 10 * 50))),
        "Foreign Born": min(100, max(0, 50 + ((diversity["Foreign Born"] - medians["Foreign Born (%)"]) / 8 * 50)))
    }
    score = sum(scores.values()) / len(scores)
    return {"Diversity": diversity, "Identity_Scores": scores, "Identity_Score": score}

def scrape_connectivity(lat, lon):
    def _scrape():
        osm_park_url = f"https://overpass-api.de/api/interpreter?data=[out:json];node(around:{RADIUS*1609.34},{lat},{lon})[leisure=park];out;"
        park_response = requests.get(osm_park_url, timeout=TIMEOUT).json()
        parks = len(park_response["elements"])
        park_access = min(50, parks * 5)
        osm_bus_url = f"https://overpass-api.de/api/interpreter?data=[out:json];node(around:{RADIUS*1609.34},{lat},{lon})[highway=bus_stop];out;"
        bus_response = requests.get(osm_bus_url, timeout=TIMEOUT).json()
        bus_stops = len(bus_response["elements"])
        print(f"Parks: {parks}, Bus Stops: {bus_stops}")
        state_fips, county_fips = get_county_fips(lat, lon)
        data = fetch_census_data(("B08301_010E", "B01001_001E"), state_fips, county_fips)
        transit_pct = float(data["B08301_010E"]) / float(data["B01001_001E"]) * 100
        metrics = {"Park Access": park_access, "Bus Stops": bus_stops, "Transit (%)": transit_pct}
        return metrics, state_fips, county_fips
    metrics, state_fips, county_fips = retry_with_backoff(_scrape)
    medians = get_nc_medians()
    scores = {
        "Park Access": min(100, max(0, 50 + ((metrics["Park Access"] - medians["Park Access"]) / 10 * 50))),
        "Bus Stops": min(100, max(0, 50 + ((metrics["Bus Stops"] - medians["Bus Stops"]) / 50 * 50))),
        "Transit (%)": min(100, max(0, 50 + ((metrics["Transit (%)"] - medians["Transit (%)"]) / 1 * 50)))
    }
    score = sum(scores.values()) / len(scores)
    return {"Park Access (%)": metrics["Park Access"], "Bus Stops": metrics["Bus Stops"], "Transit (%)": metrics["Transit (%)"], "Connectivity_Scores": scores, "Connectivity_Score": score}

def scrape_wellness(county_fips, state_fips):
    data = fetch_census_data(("B18101_001E", "B27001_028E", "B01001_001E", "C24050_026E"), state_fips, county_fips)
    total_pop = float(data["B01001_001E"])
    uninsured = float(data["B27001_028E"]) / total_pop * 100
    provider_ratio = float(data["B01001_001E"]) / float(data["C24050_026E"]) if float(data["C24050_026E"]) > 0 else 1000
    metrics = {"Uninsured (%)": uninsured, "Provider Ratio": provider_ratio}
    medians = get_nc_medians()
    scores = {
        "Uninsured (%)": min(100, max(0, 50 + ((medians["Uninsured (%)"] - uninsured) / 5 * 50))),
        "Provider Ratio": min(100, max(0, 50 + ((medians["Provider Ratio"] - provider_ratio) / 100 * 50)))
    }
    score = sum(scores.values()) / len(scores)
    return {"Obesity Proxy (%)": 0.0, "Uninsured (%)": uninsured, "Provider Ratio": provider_ratio, "Wellness_Scores": scores, "Wellness_Score": score}

def scrape_prosperity(county_fips, state_fips):
    data = fetch_census_data(("NAME", "B19013_001E", "B25003_002E", "B23025_005E", "B01001_001E"), state_fips, county_fips)
    total_pop = float(data["B01001_001E"])
    income = float(data["B19013_001E"])
    home_ownership = float(data["B25003_002E"]) / total_pop * 100
    unemployment = float(data["B23025_005E"]) / total_pop * 100
    metrics = {"Median Income ($)": income, "Home Ownership (%)": home_ownership, "Unemployment (%)": unemployment}
    medians = get_nc_medians()
    scores = {
        "Median Income ($)": min(100, max(0, 50 + ((income - medians["Median Income ($)"]) / 20000 * 50))),
        "Home Ownership (%)": min(100, max(0, 50 + ((home_ownership - medians["Home Ownership (%)"]) / 20 * 50))),
        "Unemployment (%)": min(100, max(0, 50 + ((medians["Unemployment (%)"] - unemployment) / 2 * 50)))
    }
    score = sum(scores.values()) / len(scores)
    return {"Median Income ($)": income, "Home Ownership (%)": home_ownership, "Unemployment (%)": unemployment, "Prosperity_Scores": scores, "Prosperity_Score": score}

def scrape_finance(county_fips, state_fips):
    data = fetch_census_data(("B01001_001E", "C24050_001E"), state_fips, county_fips)
    total_pop = float(data["B01001_001E"])
    biz_activity = float(data["C24050_001E"]) / total_pop * 1000
    grants_proxy = min(10, biz_activity / 50)
    metrics = {"Grants Proxy (per 1000)": grants_proxy}
    medians = get_nc_medians()
    scores = {
        "Grants Proxy (per 1000)": min(100, max(0, 50 + ((grants_proxy - medians["Grants Proxy (per 1000)"]) / 5 * 50)))
    }
    score = scores["Grants Proxy (per 1000)"]
    return {"Grants Proxy (per 1000)": grants_proxy, "Finance_Scores": scores, "Finance_Score": score}

def automate_impact_tool(address):
    print(f"Processing {address}...")
    try:
        lat, lon = geocode_address(address)
        state_fips, county_fips = get_county_fips(lat, lon)
        fetch_nc_county_data(state_fips)
        data = {
            "Identity": scrape_identity(county_fips, state_fips),
            "Connectivity": scrape_connectivity(lat, lon),
            "Wellness": scrape_wellness(county_fips, state_fips),
            "Prosperity": scrape_prosperity(county_fips, state_fips),
            "Finance": scrape_finance(county_fips, state_fips)
        }
        flat_data = {}
        for category in data:
            for key, val in data[category].items():
                flat_data[f"{category}_{key}"] = val
        df = pd.DataFrame([flat_data])
        print("Scores:", {k: v[f"{k}_Score"] for k, v in data.items()})
        df.to_csv("impact_tool_output.csv", index=False)
        print("Data saved to impact_tool_output.csv")
        print(f"Geocoded to: ({lat}, {lon})")
        return df
    except Exception as e:
        print(f"Error processing request: {e}")
        raise

def visualize_data(df):
    plt.figure(figsize=(8, 6))
    diversity = df["Identity_Diversity"].iloc[0]
    plt.pie([diversity["White"], diversity["Black"], diversity["Hispanic"], diversity["Foreign Born"]], 
            labels=["White", "Black", "Hispanic", "Foreign Born"], autopct='%1.1f%%', 
            startangle=90, colors=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])
    plt.title("Community Diversity", fontsize=14)
    plt.axis('equal')
    plt.savefig("identity_diversity.png")
    plt.close()

    plt.figure(figsize=(10, 6))
    connectivity_metrics = ["Park Access", "Bus Stops", "Transit (%)"]
    connectivity_values = [df["Connectivity_Park Access (%)"].iloc[0], 
                         df["Connectivity_Bus Stops"].iloc[0], 
                         df["Connectivity_Transit (%)"].iloc[0]]
    plt.bar(connectivity_metrics, connectivity_values, color=['#FFCC00', '#00CC66', '#0066CC'])
    plt.title("Connectivity Overview", fontsize=14)
    plt.ylabel("Value", fontsize=12)
    for i, v in enumerate(connectivity_values):
        plt.text(i, v + max(connectivity_values) * 0.05, f"{v:.1f}", ha='center')
    plt.savefig("connectivity_metrics.png")
    plt.close()

    plt.figure(figsize=(10, 6))
    wellness_metrics = ["Uninsured (%)", "Provider Ratio"]
    wellness_values = [df["Wellness_Uninsured (%)"].iloc[0], 
                      df["Wellness_Provider Ratio"].iloc[0]]
    plt.bar(wellness_metrics, wellness_values, color=['#FF33CC', '#33CCCC'])
    plt.title("Wellness Overview", fontsize=14)
    plt.ylabel("Value", fontsize=12)
    for i, v in enumerate(wellness_values):
        plt.text(i, v + max(wellness_values) * 0.05, f"{v:.1f}", ha='center')
    plt.savefig("wellness_metrics.png")
    plt.close()

    plt.figure(figsize=(10, 6))
    prosperity_metrics = ["Median Income ($K)", "Home Ownership (%)", "Unemployment (%)"]
    prosperity_values = [df["Prosperity_Median Income ($)"].iloc[0] / 1000, 
                        df["Prosperity_Home Ownership (%)"].iloc[0], 
                        df["Prosperity_Unemployment (%)"].iloc[0]]
    plt.bar(prosperity_metrics, prosperity_values, color=['#FF9900', '#0099FF', '#CC0000'])
    plt.title("Prosperity Overview", fontsize=14)
    plt.ylabel("Value", fontsize=12)
    for i, v in enumerate(prosperity_values):
        plt.text(i, v + max(prosperity_values) * 0.05, f"{v:.1f}", ha='center')
    plt.savefig("prosperity_metrics.png")
    plt.close()

    plt.figure(figsize=(6, 6))
    plt.bar(["Grants Proxy (per 1000)"], [df["Finance_Grants Proxy (per 1000)"].iloc[0]], color='#00CC99')
    plt.title("Finance Overview", fontsize=14)
    plt.ylabel("Grants per 1000", fontsize=12)
    plt.text(0, df["Finance_Grants Proxy (per 1000)"].iloc[0] + 0.5, 
             f"{df['Finance_Grants Proxy (per 1000)'].iloc[0]:.1f}", ha='center')
    plt.savefig("finance_metric.png")
    plt.close()

    plt.figure(figsize=(12, 6))
    categories = ["Identity", "Connectivity", "Wellness", "Prosperity", "Finance"]
    scores = [df["Identity_Identity_Score"].iloc[0], df["Connectivity_Connectivity_Score"].iloc[0], 
              df["Wellness_Wellness_Score"].iloc[0], df["Prosperity_Prosperity_Score"].iloc[0], 
              df["Finance_Finance_Score"].iloc[0]]
    plt.bar(categories, scores, color=['#FF9999', '#66B2FF', '#CC33FF', '#FF9900', '#00CC99'])
    plt.title("Category Impact Scores", fontsize=14)
    plt.ylabel("Score (0-100)", fontsize=12)
    plt.ylim(0, 100)
    for i, v in enumerate(scores):
        plt.text(i, v + 2, f"{v:.1f}", ha='center')
    plt.savefig("category_scores.png")
    plt.close()

    all_scores = []
    for cat in ["Identity", "Connectivity", "Wellness", "Prosperity", "Finance"]:
        scores_dict = df[f"{cat}_{cat}_Scores"].iloc[0]
        all_scores.extend(scores_dict.values())
    overall_score = sum(all_scores) / len(all_scores) if all_scores else 0
    plt.figure(figsize=(6, 6))
    plt.bar(["Overall"], [overall_score], color='#666666')
    plt.title("Overall Impact Score", fontsize=14)
    plt.ylabel("Score (0-100)", fontsize=12)
    plt.ylim(0, 100)
    plt.text(0, overall_score + 2, f"{overall_score:.1f}", ha='center')
    plt.savefig("overall_scores.png")
    plt.close()
    print("Graphs saved: identity_diversity.png, connectivity_metrics.png, wellness_metrics.png, prosperity_metrics.png, finance_metric.png, category_scores.png, overall_scores.png")

if __name__ == "__main__":
    address = input("Enter address: ")
    start_time = time.time()
    try:
        result = automate_impact_tool(address)
        print(f"Completed data collection in {time.time() - start_time:.2f} seconds")
        visualize_data(result)
    except Exception as e:
        print(f"Failed to process request: {e}")

Enter address: 123 Main St, Charlotte, NC
Processing 123 Main St, Charlotte, NC...
Geocoded 123 Main St, Charlotte, NC to (35.20854790943231, -80.8311796379137)
Geocoding API response is successful
Geocoded to State FIPS: 37, County FIPS: 119
Using existing nc_county_data.xlsx
Using Mecklenburg cache for ('NAME', 'B03002_003E', 'B03002_004E', 'B03002_012E', 'B01001_001E', 'B01002_001E', 'B05002_013E')
Calculated medians: {'White (%)': 67.3887521566493, 'Black (%)': 17.767609179573636, 'Hispanic (%)': 6.8423496082637225, 'Foreign Born (%)': 3.8914497825014154, 'Park Access': 1.4966990336264674, 'Bus Stops': 7.483495168132338, 'Transit (%)': 0.07483495168132337, 'Uninsured (%)': 3.074482984159226, 'Provider Ratio': 157.96463166891695, 'Median Income ($)': 51497.0, 'Home Ownership (%)': 28.471411394017153, 'Unemployment (%)': 2.596609445713743, 'Grants Proxy (per 1000)': 8.696259228740153}
Parks: 3, Bus Stops: 198
Geocoding API response is successful
Geocoded to State FIPS: 37, County FIP

In [2]:
import requests
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd
import numpy as np  # Added for percentile calculations
from census import Census
import time
import os
import matplotlib.pyplot as plt
import statistics
import json
import unicodedata

# Constants
CENSUS_API_KEY = "86cfe7a999f6607864204747a5ac83b7c77e02fb"
RADIUS = 5
BASE_YEAR = 2021
TIMEOUT = 30
MAX_RETRIES = 3

geolocator = Nominatim(user_agent="impact_tool")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1, max_retries=3, error_wait_seconds=5)
c = Census(CENSUS_API_KEY, year=BASE_YEAR)
census_cache = {}
mecklenburg_cache_file = "mecklenburg_cache.json"

def load_mecklenburg_cache():
    try:
        if os.path.exists(mecklenburg_cache_file):
            with open(mecklenburg_cache_file, 'r') as f:
                return json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error reading cache file: {e}. Starting with empty cache.")
        if os.path.exists(mecklenburg_cache_file):
            os.remove(mecklenburg_cache_file)
    return {}

def save_mecklenburg_cache(cache):
    try:
        with open(mecklenburg_cache_file, 'w') as f:
            json.dump(cache, f, default=str)
    except Exception as e:
        print(f"Error saving cache: {e}")

def retry_with_backoff(func, *args, max_attempts=MAX_RETRIES, base_delay=1, max_delay=10):
    for attempt in range(max_attempts):
        try:
            return func(*args)
        except (requests.RequestException, ValueError) as e:
            if attempt == max_attempts - 1:
                raise
            delay = min(base_delay * (2 ** attempt), max_delay)
            print(f"Attempt {attempt + 1} failed: {e}. Retrying after {delay}s...")
            time.sleep(delay)
    raise ValueError("Max retries reached")

def geocode_address(address):
    def _geocode():
        location = geocode(address, timeout=TIMEOUT)
        if location:
            print(f"Geocoded {address} to ({location.latitude}, {location.longitude})")
            return location.latitude, location.longitude
        raise ValueError(f"Address not found: {address}")
    return retry_with_backoff(_geocode)

def get_county_fips(lat, lon):
    def _get_fips():
        url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={lon}&y={lat}&benchmark=Public_AR_Current&vintage=Current_Current&format=json"
        response = requests.get(url, timeout=TIMEOUT)
        response.raise_for_status()
        data = response.json()
        print("Geocoding API response is successful")
        county = data["result"]["geographies"]["Counties"][0]
        state_fips = county["GEOID"][:2]
        county_fips = county["GEOID"][-3:]
        print(f"Geocoded to State FIPS: {state_fips}, County FIPS: {county_fips}")
        return state_fips, county_fips
    return retry_with_backoff(_get_fips)

def fetch_census_data(fields, state_fips, county_fips):
    cache_key = f"{county_fips}_{'_'.join(fields)}"
    meck_cache = load_mecklenburg_cache()
    if cache_key in meck_cache:
        print(f"Using Mecklenburg cache for {fields}")
        return meck_cache[cache_key]
    if cache_key in census_cache:
        print(f"Using global cache for {fields}")
        return census_cache[cache_key]
    def _fetch():
        try:
            response = c.acs5.state_county(fields, state_fips, county_fips)
            print(f"ACS5 API call successful for fields: {fields}")
            data = response[0] if response else None
            if not data:
                raise ValueError("No data returned from Census API")
            return {k: str(v) for k, v in data.items()}
        except Exception as e:
            print(f"Census library error: {e}")
            url = f"https://api.census.gov/data/{BASE_YEAR}/acs/acs5?get={','.join(fields)}&for=county:{county_fips}&in=state:{state_fips}&key={CENSUS_API_KEY}"
            response = requests.get(url, timeout=TIMEOUT)
            response.raise_for_status()
            data = response.json()
            print(f"Direct Census API response: {data}")
            data = dict(zip(data[0], data[1])) if len(data) > 1 else None
            if not data:
                raise ValueError("No data returned from direct Census API")
            return {k: str(v) for k, v in data.items()}
    data = retry_with_backoff(_fetch)
    census_cache[cache_key] = data
    if county_fips == "119" and state_fips == "37":
        meck_cache[cache_key] = data
        save_mecklenburg_cache(meck_cache)
    return data

def fetch_nc_county_data(state_fips):
    if os.path.exists("nc_county_data.xlsx"):
        print("Using existing nc_county_data.xlsx")
        return pd.read_excel("nc_county_data.xlsx")
    fields = ("NAME", "B03002_003E", "B03002_004E", "B03002_012E", "B05002_013E", 
              "B01001_001E", "B08301_010E", "B27001_028E", "C24050_026E", 
              "B19013_001E", "B25003_002E", "B23025_005E", "C24050_001E")
    nc_data = c.acs5.state_county(fields, state_fips, Census.ALL)
    county_data = []
    for d in nc_data:
        pop = float(d["B01001_001E"])
        if pop > 0:
            county = {
                "FIPS": d["state"] + d["county"],
                "Name": d["NAME"],
                "White (%)": float(d["B03002_003E"]) / pop * 100,
                "Black (%)": float(d["B03002_004E"]) / pop * 100,
                "Hispanic (%)": float(d["B03002_012E"]) / pop * 100,
                "Foreign Born (%)": float(d["B05002_013E"]) / pop * 100,
                "Transit (%)": float(d["B08301_010E"]) / pop * 100,
                "Uninsured (%)": float(d["B27001_028E"]) / pop * 100,
                "Provider Ratio": pop / float(d["C24050_026E"]) if float(d["C24050_026E"]) > 0 else 1000,
                "Median Income ($)": float(d["B19013_001E"]),
                "Home Ownership (%)": float(d["B25003_002E"]) / pop * 100,
                "Unemployment (%)": float(d["B23025_005E"]) / pop * 100,
                "Grants Proxy (per 1000)": min(10, (float(d["C24050_001E"]) / pop * 1000) / 50)
            }
            county["Park Access"] = min(50, county["Transit (%)"] * 20)
            county["Bus Stops"] = county["Transit (%)"] * 100
            county_data.append(county)
    df = pd.DataFrame(county_data)
    df.to_excel("nc_county_data.xlsx", index=False)
    print("Saved county data to nc_county_data.xlsx")
    return df

def get_nc_medians():
    try:
        df = pd.read_excel("nc_county_data.xlsx")
        medians = {
            "White (%)": df["White (%)"].median(),
            "Black (%)": df["Black (%)"].median(),
            "Hispanic (%)": df["Hispanic (%)"].median(),
            "Foreign Born (%)": df["Foreign Born (%)"].median(),
            "Park Access": df["Park Access"].median(),
            "Bus Stops": df["Bus Stops"].median(),
            "Transit (%)": df["Transit (%)"].median(),
            "Uninsured (%)": df["Uninsured (%)"].median(),
            "Provider Ratio": df["Provider Ratio"].median(),
            "Median Income ($)": df["Median Income ($)"].median(),
            "Home Ownership (%)": df["Home Ownership (%)"].median(),
            "Unemployment (%)": df["Unemployment (%)"].median(),
            "Grants Proxy (per 1000)": df["Grants Proxy (per 1000)"].median()
        }
        print(f"Calculated medians: {medians}")
        return medians
    except Exception as e:
        print(f"Error reading Excel: {e}, using fallback medians")
        return {
            "White (%)": 60, "Black (%)": 20, "Hispanic (%)": 8, "Foreign Born (%)": 7,
            "Park Access": 10, "Bus Stops": 50, "Transit (%)": 0.5,
            "Uninsured (%)": 5, "Provider Ratio": 150,
            "Median Income ($)": 55000, "Home Ownership (%)": 40, "Unemployment (%)": 4,
            "Grants Proxy (per 1000)": 7
        }

def clean_string(text):
    """Clean string to ensure UTF-8 compatibility."""
    if not isinstance(text, str):
        text = str(text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    return text

def calculate_all_metrics(state_fips):
    # Load NC county data
    try:
        df = fetch_nc_county_data(state_fips)
    except Exception as e:
        print(f"Error fetching county data: {e}")
        return

    # Convert FIPS to string to handle indexing
    df["FIPS"] = df["FIPS"].astype(str)

    # Get NC medians
    medians = get_nc_medians()

    # Calculate Winsorized Min/Max (5th and 95th percentiles)
    mins = {
        "White (%)": np.percentile(df["White (%)"], 5),
        "Black (%)": np.percentile(df["Black (%)"], 5),
        "Hispanic (%)": np.percentile(df["Hispanic (%)"], 5),
        "Foreign Born (%)": np.percentile(df["Foreign Born (%)"], 5),
        "Park Access": np.percentile(df["Park Access"], 5),
        "Bus Stops": np.percentile(df["Bus Stops"], 5),
        "Transit (%)": np.percentile(df["Transit (%)"], 5),
        "Uninsured (%)": np.percentile(df["Uninsured (%)"], 5),
        "Provider Ratio": np.percentile(df["Provider Ratio"], 5),
        "Median Income ($)": np.percentile(df["Median Income ($)"], 5),
        "Home Ownership (%)": np.percentile(df["Home Ownership (%)"], 5),
        "Unemployment (%)": np.percentile(df["Unemployment (%)"], 5),
        "Grants Proxy (per 1000)": np.percentile(df["Grants Proxy (per 1000)"], 5)
    }
    maxs = {
        "White (%)": np.percentile(df["White (%)"], 95),
        "Black (%)": np.percentile(df["Black (%)"], 95),
        "Hispanic (%)": np.percentile(df["Hispanic (%)"], 95),
        "Foreign Born (%)": np.percentile(df["Foreign Born (%)"], 95),
        "Park Access": np.percentile(df["Park Access"], 95),
        "Bus Stops": np.percentile(df["Bus Stops"], 95),
        "Transit (%)": np.percentile(df["Transit (%)"], 95),
        "Uninsured (%)": np.percentile(df["Uninsured (%)"], 95),
        "Provider Ratio": np.percentile(df["Provider Ratio"], 95),
        "Median Income ($)": np.percentile(df["Median Income ($)"], 95),
        "Home Ownership (%)": np.percentile(df["Home Ownership (%)"], 95),
        "Unemployment (%)": np.percentile(df["Unemployment (%)"], 95),
        "Grants Proxy (per 1000)": np.percentile(df["Grants Proxy (per 1000)"], 95)
    }

    # Calculate state scores (normalize the medians)
    state_scores = {}
    for metric in medians:
        clipped_value = np.clip(medians[metric], mins[metric], maxs[metric])
        if maxs[metric] == mins[metric]:  # Avoid division by zero
            normalized_score = 50.0  # If min == max, assign a neutral score
        else:
            normalized_score = (clipped_value - mins[metric]) / (maxs[metric] - mins[metric]) * 100
        # Adjust for directionality
        if metric in ["White (%)", "Uninsured (%)", "Provider Ratio", "Unemployment (%)"]:  # Lower is better
            state_scores[f"{metric} Score"] = 100 - normalized_score
        else:  # Higher is better
            state_scores[f"{metric} Score"] = normalized_score

    # Initialize results
    results = []

    # Process each county
    for _, row in df.iterrows():
        county_name = clean_string(row["Name"])
        county_fips = row["FIPS"][-3:] if len(row["FIPS"]) >= 5 else row["FIPS"]

        # Identity Metrics
        identity = {
            "White (%)": float(row["White (%)"]) if pd.notna(row["White (%)"]) else 0.0,
            "Black (%)": float(row["Black (%)"]) if pd.notna(row["Black (%)"]) else 0.0,
            "Hispanic (%)": float(row["Hispanic (%)"]) if pd.notna(row["Hispanic (%)"]) else 0.0,
            "Foreign Born (%)": float(row["Foreign Born (%)"]) if pd.notna(row["Foreign Born (%)"]) else 0.0
        }
        identity_scores = {}
        identity_diffs = {}
        for metric in identity:
            clipped_value = np.clip(identity[metric], mins[metric], maxs[metric])
            if maxs[metric] == mins[metric]:  # Avoid division by zero
                score = 50.0
            else:
                score = (clipped_value - mins[metric]) / (maxs[metric] - mins[metric]) * 100
            if metric == "White (%)":  # Lower is better
                score = 100 - score
            identity_scores[f"{metric} Score"] = score
            identity_diffs[f"{metric} Diff"] = round(score - state_scores[f"{metric} Score"], 2)

        identity_score = sum(identity_scores.values()) / len(identity_scores) if identity_scores else 0
        state_identity_score = sum(state_scores[f"{metric} Score"] for metric in identity) / len(identity)
        identity_diff = round(identity_score - state_identity_score, 2)

        # Connectivity Metrics
        connectivity = {
            "Park Access": float(row["Park Access"]) if pd.notna(row["Park Access"]) else 0.0,
            "Bus Stops": float(row["Bus Stops"]) if pd.notna(row["Bus Stops"]) else 0.0,
            "Transit (%)": float(row["Transit (%)"]) if pd.notna(row["Transit (%)"]) else 0.0
        }
        connectivity_scores = {}
        connectivity_diffs = {}
        for metric in connectivity:
            clipped_value = np.clip(connectivity[metric], mins[metric], maxs[metric])
            if maxs[metric] == mins[metric]:
                score = 50.0
            else:
                score = (clipped_value - mins[metric]) / (maxs[metric] - mins[metric]) * 100
            connectivity_scores[f"{metric} Score"] = score
            connectivity_diffs[f"{metric} Diff"] = round(score - state_scores[f"{metric} Score"], 2)

        connectivity_score = sum(connectivity_scores.values()) / len(connectivity_scores) if connectivity_scores else 0
        state_connectivity_score = sum(state_scores[f"{metric} Score"] for metric in connectivity) / len(connectivity)
        connectivity_diff = round(connectivity_score - state_connectivity_score, 2)

        # Wellness Metrics
        wellness = {
            "Uninsured (%)": float(row["Uninsured (%)"]) if pd.notna(row["Uninsured (%)"]) else 0.0,
            "Provider Ratio": float(row["Provider Ratio"]) if pd.notna(row["Provider Ratio"]) else 0.0
        }
        wellness_scores = {}
        wellness_diffs = {}
        for metric in wellness:
            clipped_value = np.clip(wellness[metric], mins[metric], maxs[metric])
            if maxs[metric] == mins[metric]:
                score = 50.0
            else:
                score = (clipped_value - mins[metric]) / (maxs[metric] - mins[metric]) * 100
            if metric in ["Uninsured (%)", "Provider Ratio"]:  # Lower is better
                score = 100 - score
            wellness_scores[f"{metric} Score"] = score
            wellness_diffs[f"{metric} Diff"] = round(score - state_scores[f"{metric} Score"], 2)

        wellness_score = sum(wellness_scores.values()) / len(wellness_scores) if wellness_scores else 0
        state_wellness_score = sum(state_scores[f"{metric} Score"] for metric in wellness) / len(wellness)
        wellness_diff = round(wellness_score - state_wellness_score, 2)

        # Prosperity Metrics
        prosperity = {
            "Median Income ($)": float(row["Median Income ($)"]) if pd.notna(row["Median Income ($)"]) else 0.0,
            "Home Ownership (%)": float(row["Home Ownership (%)"]) if pd.notna(row["Home Ownership (%)"]) else 0.0,
            "Unemployment (%)": float(row["Unemployment (%)"]) if pd.notna(row["Unemployment (%)"]) else 0.0
        }
        prosperity_scores = {}
        prosperity_diffs = {}
        for metric in prosperity:
            clipped_value = np.clip(prosperity[metric], mins[metric], maxs[metric])
            if maxs[metric] == mins[metric]:
                score = 50.0
            else:
                score = (clipped_value - mins[metric]) / (maxs[metric] - mins[metric]) * 100
            if metric == "Unemployment (%)":  # Lower is better
                score = 100 - score
            prosperity_scores[f"{metric} Score"] = score
            prosperity_diffs[f"{metric} Diff"] = round(score - state_scores[f"{metric} Score"], 2)

        prosperity_score = sum(prosperity_scores.values()) / len(prosperity_scores) if prosperity_scores else 0
        state_prosperity_score = sum(state_scores[f"{metric} Score"] for metric in prosperity) / len(prosperity)
        prosperity_diff = round(prosperity_score - state_prosperity_score, 2)

        # Finance Metrics
        finance = {
            "Grants Proxy (per 1000)": float(row["Grants Proxy (per 1000)"]) if pd.notna(row["Grants Proxy (per 1000)"]) else 0.0
        }
        finance_scores = {}
        finance_diffs = {}
        for metric in finance:
            clipped_value = np.clip(finance[metric], mins[metric], maxs[metric])
            if maxs[metric] == mins[metric]:
                score = 50.0
            else:
                score = (clipped_value - mins[metric]) / (maxs[metric] - mins[metric]) * 100
            finance_scores[f"{metric} Score"] = score
            finance_diffs[f"{metric} Diff"] = round(score - state_scores[f"{metric} Score"], 2)

        finance_score = finance_scores["Grants Proxy (per 1000) Score"]
        state_finance_score = state_scores["Grants Proxy (per 1000) Score"]
        finance_diff = round(finance_score - state_finance_score, 2)

        # Overall Score
        all_scores = list(identity_scores.values()) + list(connectivity_scores.values()) + list(wellness_scores.values()) + list(prosperity_scores.values()) + list(finance_scores.values())
        overall_score = sum(all_scores) / len(all_scores) if all_scores else 0
        all_state_scores = list(state_scores.values())
        overall_state_score = sum(all_state_scores) / len(all_state_scores) if all_state_scores else 0
        overall_diff = round(overall_score - overall_state_score, 2)

        # Append flattened results
        result = {
            "County": county_name,
            "FIPS": row["FIPS"],
            "White (%)": identity["White (%)"],
            "Black (%)": identity["Black (%)"],
            "Hispanic (%)": identity["Hispanic (%)"],
            "Foreign Born (%)": identity["Foreign Born (%)"],
            "White Score": round(identity_scores["White (%) Score"], 2),
            "Black Score": round(identity_scores["Black (%) Score"], 2),
            "Hispanic Score": round(identity_scores["Hispanic (%) Score"], 2),
            "Foreign Born Score": round(identity_scores["Foreign Born (%) Score"], 2),
            "White Diff": identity_diffs["White (%) Diff"],
            "Black Diff": identity_diffs["Black (%) Diff"],
            "Hispanic Diff": identity_diffs["Hispanic (%) Diff"],
            "Foreign Born Diff": identity_diffs["Foreign Born (%) Diff"],
            "Identity Score": round(identity_score, 2),
            "Identity Diff": identity_diff,
            "Park Access (%)": connectivity["Park Access"],
            "Bus Stops": connectivity["Bus Stops"],
            "Transit (%)": connectivity["Transit (%)"],
            "Park Access Score": round(connectivity_scores["Park Access Score"], 2),
            "Bus Stops Score": round(connectivity_scores["Bus Stops Score"], 2),
            "Transit Score": round(connectivity_scores["Transit (%) Score"], 2),
            "Park Access Diff": connectivity_diffs["Park Access Diff"],
            "Bus Stops Diff": connectivity_diffs["Bus Stops Diff"],
            "Transit Diff": connectivity_diffs["Transit (%) Diff"],
            "Connectivity Score": round(connectivity_score, 2),
            "Connectivity Diff": connectivity_diff,
            "Uninsured (%)": wellness["Uninsured (%)"],
            "Provider Ratio": wellness["Provider Ratio"],
            "Uninsured Score": round(wellness_scores["Uninsured (%) Score"], 2),
            "Provider Ratio Score": round(wellness_scores["Provider Ratio Score"], 2),
            "Uninsured Diff": wellness_diffs["Uninsured (%) Diff"],
            "Provider Ratio Diff": wellness_diffs["Provider Ratio Diff"],
            "Wellness Score": round(wellness_score, 2),
            "Wellness Diff": wellness_diff,
            "Median Income ($)": prosperity["Median Income ($)"],
            "Home Ownership (%)": prosperity["Home Ownership (%)"],
            "Unemployment (%)": prosperity["Unemployment (%)"],
            "Median Income Score": round(prosperity_scores["Median Income ($) Score"], 2),
            "Home Ownership Score": round(prosperity_scores["Home Ownership (%) Score"], 2),
            "Unemployment Score": round(prosperity_scores["Unemployment (%) Score"], 2),
            "Median Income Diff": prosperity_diffs["Median Income ($) Diff"],
            "Home Ownership Diff": prosperity_diffs["Home Ownership (%) Diff"],
            "Unemployment Diff": prosperity_diffs["Unemployment (%) Diff"],
            "Prosperity Score": round(prosperity_score, 2),
            "Prosperity Diff": prosperity_diff,
            "Grants Proxy (per 1000)": finance["Grants Proxy (per 1000)"],
            "Grants Proxy Score": round(finance_scores["Grants Proxy (per 1000) Score"], 2),
            "Grants Proxy Diff": finance_diffs["Grants Proxy (per 1000) Diff"],
            "Finance Score": round(finance_score, 2),
            "Finance Diff": finance_diff,
            "Overall Score": round(overall_score, 2),
            "Overall Diff": overall_diff
        }
        results.append(result)

    # Create DataFrame
    result_df = pd.DataFrame(results)
    
    # Replace any NaN values
    result_df = result_df.fillna(0.0)
    
    # Sort by Overall Score (descending)
    result_df = result_df.sort_values(by="Overall Score", ascending=False)
    
    # Save to CSV for Tableau
    output_file = "nc_all_county_metrics_winsorized.csv"
    try:
        result_df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"All county metrics saved to {output_file} for Tableau visualization")
    except Exception as e:
        print(f"Error saving to CSV: {e}")

if __name__ == "__main__":
    # Run for North Carolina (state FIPS 37)
    calculate_all_metrics("37")

Using existing nc_county_data.xlsx
Calculated medians: {'White (%)': 67.3887521566493, 'Black (%)': 17.767609179573636, 'Hispanic (%)': 6.8423496082637225, 'Foreign Born (%)': 3.8914497825014154, 'Park Access': 1.4966990336264674, 'Bus Stops': 7.483495168132338, 'Transit (%)': 0.07483495168132337, 'Uninsured (%)': 3.074482984159226, 'Provider Ratio': 157.96463166891695, 'Median Income ($)': 51497.0, 'Home Ownership (%)': 28.471411394017153, 'Unemployment (%)': 2.596609445713743, 'Grants Proxy (per 1000)': 8.696259228740153}
All county metrics saved to nc_all_county_metrics_winsorized.csv for Tableau visualization


In [3]:
# Import the required library
import pandas as pd

# Step 1: Load the dataset
# Replace 'nc_all_county_metrics_winsorized.csv' with the path to your file if it's not in the same directory
df = pd.read_csv('nc_all_county_metrics_winsorized.csv')

# Step 2: Display the first few rows to inspect the data
print("Initial Data Preview:")
print(df.head())

# Step 3: Clean the 'County' column by removing quotes
# The County column has values like "Durham County, North Carolina" with quotes, which we need to remove
df['County'] = df['County'].str.strip('"')

# Step 4: Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

# Step 5: Ensure proper data types
# Most columns should be numeric (floats/ints), except for 'County' and 'FIPS' which should be strings
# Convert 'FIPS' to string to preserve leading zeros (important for Tableau's geographic mapping)
df['FIPS'] = df['FIPS'].astype(str).str.zfill(5)  # Ensure FIPS is 5 digits with leading zeros

# Verify data types
print("\nData Types:")
print(df.dtypes)

# Step 6: Check for any duplicate rows
print("\nNumber of Duplicate Rows:", df.duplicated().sum())

# Step 7: Save the cleaned dataset to a new CSV file for Tableau
cleaned_file_path = 'nc_county_metrics_cleaned.csv'
df.to_csv(cleaned_file_path, index=False)
print(f"\nCleaned dataset saved to: {cleaned_file_path}")

# Step 8: Display the first few rows of the cleaned dataset
print("\nCleaned Data Preview:")
print(df.head())

Initial Data Preview:
                               County   FIPS  White (%)  Black (%)  \
0       Durham County, North Carolina  37063  42.564642  34.825361   
1  Mecklenburg County, North Carolina  37119  45.586766  31.008171   
2     Cabarrus County, North Carolina  37025  62.563291  18.609855   
3         Wake County, North Carolina  37183  58.803127  19.434118   
4       Orange County, North Carolina  37135  68.900635  10.743269   

   Hispanic (%)  Foreign Born (%)  White Score  Black Score  Hispanic Score  \
0     13.704060         14.310971        90.94        69.80           94.41   
1     13.675585         15.951095        85.33        61.94           94.16   
2     10.997288          8.523960        53.78        36.40           70.87   
3     10.310967         13.479045        60.77        38.10           64.91   
4      8.600451         12.597031        42.01        20.19           50.04   

   Foreign Born Score  ...  Unemployment Diff  Prosperity Score  \
0              